In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA

# Google data
### Write more

In [2]:
google_data=pd.read_pickle('Google_Alibaba.pkl')
google_data=google_data['maxCPU_usage']
endvalue=google_data.shape[0]
google_data.head()

20244     0.2009
164605    0.1049
308807    0.1226
462097    0.1602
618227    0.1914
Name: maxCPU_usage, dtype: object

# Alibaba Data
### Write more

In [3]:
alibaba_data=pd.read_pickle('Alibaba_Google.pkl')
alibaba_data=alibaba_data[0:9654]
alibaba_data.tail()

,0
9649,0.000081
9650,0.000046
9651,0.000047
9652,0.000047
9653,0.000043


In [4]:
google_data=np.array(google_data,dtype=float)
google_min=np.amin(google_data)
google_data.shape

(9654,)

In [5]:
alibaba_data=np.array(alibaba_data,dtype=float)
alibaba_data=np.ravel(alibaba_data)
alibaba_min=np.amin(alibaba_data)
alibaba_data.shape

(9654,)

## Removing the NaN values

In [6]:
alibaba_data=np.nan_to_num(alibaba_data,nan=alibaba_min)
google_data=np.nan_to_num(google_data,nan=google_min)

## Combining the data

In [7]:
combined=np.append(google_data,alibaba_data)

## Generating Training and Test data
### Train and Test data is generated according to the shape of the traces

In [8]:
train= combined[0:endvalue]

## Function to get valid list

In [9]:
def try_forecast_model(p,q,d):
    temp=[]
    model=ARIMA(train,order=(p,q,d))
    try:
        model_fit=model.fit(disp=0)
    except:
        temp.append(p)
        temp.append(q)
        temp.append(d)
        invalid_list.append(temp)

## Declaring 'p','q','d' values for the model
### These values are being declared to do a sort of grid search

In [10]:
AR_parameter=[2]
MA_parameter=[2]
difference=[4]

In [11]:
total_list=[]
invalid_list=[]
for p in AR_parameter:
    for q in MA_parameter:
        for d in difference:
            initial_temp=[]
            initial_temp.append(p)
            initial_temp.append(q)
            initial_temp.append(d)
            total_list.append(initial_temp)
            try_forecast_model(p,q,d)

/home/abhilash/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/abhilash/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


## Finding a valid list of hyperparameters

In [12]:
valid_list = []
for list in total_list:
    if list not in invalid_list:
        valid_list.append(list) 

In [13]:
valid_list

[[2, 2, 4]]

## Forecasting Model

In [14]:
rmse_accuracy=[]
predicted=[]
mape=[]
def forecast_model(p,q,d):
    temp=[]
    model=ARIMA(train,order=(p,q,d))
    model_fit=model.fit(disp=0)
    end_values=[10,15,50,100]
    for value in end_values:
        forecast=model_fit.predict(start=endvalue,end=endvalue+value)
        predicted.append(forecast)
        actual_values=combined[endvalue:(endvalue+value+1)]
        accuracy=np.mean((forecast - actual_values)**2)**.5
        rmse_accuracy.append(accuracy)
        acc = np.mean(np.abs(forecast - actual_values)/np.abs(actual_values))
        mape.append(acc)

### Running the model

In [15]:
for val in valid_list:
    forecast_model(val[0],val[1],val[2])

/home/abhilash/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/abhilash/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [16]:
rmse_accuracy

[0.028087740274442018,
 0.0232891757930113,
 0.01304464061991965,
 0.009269624333946657]

In [17]:
mape

[211.16499622898408, 145.44538077911224, 46.30934162652635, 23.878855517614262]

In [19]:
maximum=np.amax(rmse_accuracy)

In [20]:
ind=0
for i in range(0,len(rmse_accuracy)):
    if rmse_accuracy[i] == maximum:
        ind=i
        break

In [21]:
ind

44

In [22]:
mape[ind]

211.16499622898408

In [23]:
rmse_accuracy[ind]

0.028087740274442018

In [26]:
valid_list[11]

[2, 2, 4]

In [27]:
np.save('GoogleAlibaba_predicted.npy',predicted[ind])